###### Script that merges model results with different configurations into an Excel File
** **

In [50]:
import xlsxwriter
import os
from os import listdir
from os.path import isfile
from os.path import join

In [ ]:
workbook = xlsxwriter.Workbook('../results/merged.xlsx')
worksheet = workbook.add_worksheet()
longest_width_B = 10
longest_width_CD = 10
text = 'Number of Topics'
longest_width_A = len(text)
cell_format = workbook.add_format({'bold': True})
cell_format.set_align('center')
worksheet.write('A1', text, cell_format)
worksheet.write('B1', 'Hyperparameter', cell_format)
worksheet.write('C1', 'Perplexity', cell_format)
worksheet.write('D1', 'Coherence Score', cell_format)

# iterate over directory's
for base, dirs, files in os.walk('../results/'):
    file_counter = 1
    for directory in dirs:
        dict_counter = file_counter + 1
        worksheet.write(('A' + str(dict_counter)), directory.split('_')[1])
        # iterate over files in directory
        path = f'../results/{directory}/'
        files = [f for f in listdir(path) if isfile(join(path, f))]
        for file in files:
            if file.endswith('.html') or file.endswith('.txt') or file.endswith('.csv'):
                None
            else:
                file_counter += 1
                text = file.split('_')[4] + file.split('_')[5]
                width_B = len(text)
                if width_B > longest_width_B:
                    longest_width_B = width_B
                worksheet.write(('B' + str(file_counter)), text)
                txt_path = path + file + '_scores.txt'
                with open(txt_path) as f:
                    lines = f.readlines()
                    f.close()
                for i, line in enumerate(lines):
                    text = (line.split(': ')[1]).strip()
                    width_CD = len(text)
                    if width_CD > longest_width_CD:
                        longest_width_CD = width_CD
                    if i == 0:
                        worksheet.write(('C' + str(file_counter)), float(text))
                    elif i == 1:
                        worksheet.write(('D' + str(file_counter)), float(text))
    # set column width
    worksheet.set_column(0, 0, longest_width_A)  # column A
    worksheet.set_column(1, 1, longest_width_B)  # column B
    worksheet.set_column(2, 3, longest_width_CD)  # column C & D

    # conditional formatting to find the best variant in excel file
    # > lower the perplexity better the model.
    # > higher the topic coherence, the topic is more human interpretable.
    worksheet.conditional_format(('C1:C' + str(file_counter)),{'type': '3_color_scale', 'min_color':'#88cc00', 'mid_color':'#ffcc00', 'max_color':'#ff471a'})  # > Perplexity
    worksheet.conditional_format(('D1:D' + str(file_counter)),{'type': '3_color_scale', 'min_color':'#aaff00', 'mid_color':'#99e600', 'max_color':'#88cc00'})  # > Coherence Score
workbook.close()